## Creating sub-agent 1: Fundamental Analyst agent

In this folder we will create the first sub-agent for getting our `input statements`, `balance sheets` and `cash flow statements`. This code defines an agent for retrieving income statements for publicly traded companies through a financial API service.

This `fundamental analyst agent` will be associated with an action group which will have access to three functions. Each of the function is responsible for performing certain tasks related to getting input statements of a company's stock based on the `ticker` symbol, balance sheets and other cash flow details. 

### Prerequisites
---

You are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

Before running this notebook, make sure to populate the `.env` file with your `FINANCIAL_DATASETS_API_KEY` and `TAVILY_API_KEY`. These two `API` keys will be used by the agent to access information on user related questions. You can create an `API` here: https://www.financialdatasets.ai/

In [ ]:
# Uninstall the current version of boto3, botocore and awscli, to install the updated version of boto3
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

### Restart kernel

If you face issues to apply the latest multi-agent capabilities, uncomment this line to restart kernel to ensure packages updates to take effect

In [ ]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
!pip freeze | grep boto3

### Importing helper functions

On following section, we're adding `agents.py` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `agent.py`.

Those files contain helper classes totally focused on make labs experience smoothly.

All interactions with Bedrock will be handled by these classes.

#### Following are methods that you're going to invoke on this lab:

- `create_agent`: Create a new agent and respective IAM roles

- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent

- `create_agent_alias`: Create an alias for this agent

- `invoke`: Execute agent

In [ ]:
import boto3
import os
import json
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

In [ ]:
import os
import sys
import time
import json
import boto3
import shutil
import logging
import zipfile
import subprocess
from dotenv import load_dotenv

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from utils import *
from globals import *
from utils.utils import *

In [ ]:
# Load the environment variables that are defined in the ".env" file. This contains the 
# financial data API key that will enable the user to access the data
load_dotenv

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Defining agent configuration
---

In [ ]:
agent_instruction = """You are a comprehensive fundamental analyst assistant that helps users analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements.
You require the user to provide a stock ticker symbol to analyze the company's financials.

You can perform the following types of analysis by calling the functions below:

1. Income Statement Analysis:
   - Revenue growth, gross profit, operating income, net income, earnings per share, and revenue/expense breakdown

2. Balance Sheet Analysis:
   - Asset composition (current and non-current), liabilities (current and non-current), shareholders' equity, retained earnings, and financial ratios (current ratio, debt-to-equity)

3. Cash Flow Statement Analysis:
   - Net cash flow from operations, capital expenditures, business acquisitions, issuance/repayment of debt, dividends, and changes in cash and equivalents

IMPORTANT: Always use the financial dataset API you have access to to call these functions and retrieve the data to answer the user question

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above. Always give information on where you got the data to answer the user question
and do not redact anything in your response.
"""

agent_description = "Agent for comprehensive financial statement analysis including income statements, balance sheets, and cash flows"

In [ ]:
# This is the function definition for the lambda function that will be invoked
# as a part of the action group
functions = [{
    'name': 'get_income_statements',
    'description': 'Get income statements for a company',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "period": {
            "description": "period of statements (ttm, quarterly, or annual)",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of statements to retrieve",
            "required": True,
            "type": "integer"
        }
    }
},
{
    'name': 'get_balance_sheets',
    'description': 'Get balance sheets for a company',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "period": {
            "description": "period of statements (ttm, quarterly, or annual)",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of statements to retrieve",
            "required": True,
            "type": "integer"
        }
    }
},
{
    'name': 'get_cash_flow_statements',
    'description': 'Get cash flow statements for a company',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "period": {
            "description": "period of statements (ttm, quarterly, or annual)",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of statements to retrieve",
            "required": True,
            "type": "integer"
        }
    }
}]

### Add a lambda layer to the lambda function
---

In this section of the notebook, we will add a lambda layer to the lambda function. We will be installing `requests` to make requests to the financial data API key.

In [ ]:
# Create and publish the layer
# In this case we want to add a layer to the lambda containing files to import the requests library
layer_zip = create_lambda_layer(['requests'])
layer_arn = publish_layer('fundamental-agent-lambda-layer')

### Create agent
---

Next we will create the agent with the provided information. In this, we will use the prompt configuration, the lambda function and lambda layers, update the lambda function to contain the `FINANCIAL_DATASETS_API_KEY` that the agent can use to fetch data to user inputs, and then finally create the agent.

In [ ]:
# Create the fundamental analyst agent
fundamental_analyst_agent = agents.create_agent(
    SUB_AGENT_FUNDAMENTAL_ANALYST,
    agent_description,
    agent_instruction,
    BEDROCK_MODEL_CLAUDE_HAIKU,
)

In [ ]:
# Add the action group with lambda to this fundamental analyst agent
agents.add_action_group_with_lambda(
    agent_name=SUB_AGENT_FUNDAMENTAL_ANALYST,
    lambda_function_name=f'{SUB_AGENT_FUNDAMENTAL_ANALYST}-lambda',
    source_code_file=FUNDAMENTAL_LAMBDA_FUNCTION_NAME,
    agent_functions=functions,
    agent_action_group_name="FundamentalAnalysisActionGroup",
    agent_action_group_description="Action group to analyze company financial statements using income statement data",
    lambda_layers=[layer_arn]
)

# Create a Lambda client and attach the API key as env variable to the lambda function
lambda_client = boto3.client('lambda')
lambda_function_name = f'{SUB_AGENT_FUNDAMENTAL_ANALYST}-lambda'
environment_variables = {
    'FINANCIAL_DATASETS_API_KEY': os.getenv('FINANCIAL_DATASETS_API_KEY')
}

In [ ]:
# Update the Lambda function's configuration to include the environment variables
# In the case of this agent, we need the financial dataset API as an env variable passed to the lambda
# so that it can be used in fetching data based on user questions
response = lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Environment={
        'Variables': environment_variables
    },
    Layers=[layer_arn]
)

### Testing the Fundamental Analyst agent
---

Now that we've created the agent, let's test it by using our `invoke_agent_helper` function. Here, we will invoke the `fundamental analyst` agent to provide balance sheet information, income statements and cash flow information based on the question provided by the user.

Since the `invoke_agent_helper` is wrapped with a `weave` decorator, it will log the input/outputs or any errors to your weave account dashboard. To create a weave API key, refer to the following link: https://wandb.ai/site/weave/

In [ ]:
# sleep for 30 seconds before invoking the agent
time.sleep(30)

In [ ]:
%%time
response = agents.invoke(
    """Summarize income statements for AAPL in 2023""", 
    fundamental_analyst_agent[0], enable_trace=True
)
print("====================")
print(response)

As seen from above, the agent rationalizes and uses the `get_income_statements` from the `FundamentalAnalysisActionGroup` to get the summary of the income statements for `AAPL`. View the API usage of the financial dataset website below that the agent calls to get the relevant data to answer the user question.

![api-usage](images/api-usage.png)

In [ ]:
%%time
response = agents.invoke(
    """What is AAPL's balance sheet information for the first quarter of May, 2021?""",
    fundamental_analyst_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
%%time
response = agents.invoke(
    """Describe AAPL's cash flow for 2024?""",
    fundamental_analyst_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
%%time
response = agents.invoke(
    """What is a llama?""",
    fundamental_analyst_agent[0], enable_trace=True
)
print("====================")
print(response)

### Create alias

For multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional.

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
fundamental_analyst_agent_id, fundamental_analyst_agent_alias_arn = agents.create_agent_alias(
    fundamental_analyst_agent[0], 'v1'
)

In [ ]:
fundamental_analyst_agent_arn = agents.get_agent_arn_by_name(SUB_AGENT_FUNDAMENTAL_ANALYST)

### Saving the information
---
Let's now save some information so that we can use this agent as a sub-agent of our financial hedge fund assistant

In [ ]:
%store fundamental_analyst_agent_id
%store fundamental_analyst_agent_alias_arn
%store fundamental_analyst_agent_arn